# Анализ сервиса вопросов и ответов о программировании

### Описание проекта

База данных StackOverflow — сервиса вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам. Данные о постах за 2008 год.

**Задача:** проанализировать ключевые метрики сервиса опросов и ответов о программировании.

#### Описание данных

- `stackoverflow.badges` — хранит информацию о значках, которые присуждаются за разные достижения;
- `stackoverflow.post_types` — хранит информацию о типе постов;
- `stackoverflow.posts` — содержит информацию о постах;
- `stackoverflow.users` — содержит информацию о пользователях;
- `stackoverflow.vote_types` — содержит информацию о типах голосов;
- `stackoverflow.votes` — содержит информацию о голосах за посты. 

### Первая часть

1. Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT 
    COUNT(id)
FROM stackoverflow.posts
WHERE post_type_id = 1 AND score > 300 OR favorites_count >= 100;

2. Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
SELECT ROUND(AVG(count),0)
FROM
(SELECT DATE_TRUNC('day', creation_date)AS date, count(id) AS count
FROM stackoverflow.posts
WHERE post_type_id = 1 AND (DATE_TRUNC('day', creation_date) BETWEEN '2008-11-01' AND '2008-11-18')
GROUP BY 1) dt;

3. 
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users AS u
LEFT JOIN stackoverflow.badges AS b ON u.id=b.user_id
WHERE CAST(u.creation_date AS date) = CAST(b.creation_date AS date);

4. Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON p.user_id=u.id
JOIN stackoverflow.votes AS v ON v.post_id=p.id
WHERE display_name ='Joel Coehoorn'

5. Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT * , ROW_NUMBER() OVER(ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id;

6. Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT u.id, count(v.id)
FROM stackoverflow.vote_types AS vp
JOIN stackoverflow.votes AS v ON v.vote_type_id=vp.id
JOIN stackoverflow.users AS u ON v.user_id=u.id
WHERE name = 'Close'
GROUP BY u.id 
ORDER BY 2 DESC, 1 DESC
LIMIT 10;

7. Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.

Отобразите несколько полей
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.

Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.  :

In [ ]:
SELECT user_id AS user, count(id) AS count, DENSE_RANK() OVER(ORDER BY count(id) DESC)
FROM stackoverflow.badges
WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id 
LIMIT 10;

8. Сколько в среднем очков получает пост каждого пользователя?

Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.  

In [ ]:
SELECT title, user_id, score, ROUND(AVG(score) OVER (PARTITION BY user_id),0)
FROM stackoverflow.posts
WHERE title !='' AND score !=0;

9. Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
SELECT title
FROM
    (SELECT user_id, COUNT(b.id) as count_1
    FROM stackoverflow.badges AS b
    JOIN stackoverflow.users AS u ON u.id=b.user_id
    GROUP BY user_id
    HAVING COUNT(b.id) > 1000) t_1
JOIN stackoverflow.posts AS p ON t_1.user_id=p.user_id
WHERE title !='';

10. Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:

- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу.

In [ ]:
SELECT id, views,
CASE
WHEN views >= 350 THEN 1
WHEN 100 <= views AND views < 350 THEN 2
WHEN views < 100 THEN 3
END
FROM stackoverflow.users
WHERE views > 0 AND location LIKE '%Canada%';

11. Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH u AS
(SELECT id, views,
CASE
WHEN views >= 350 THEN 1
WHEN 100 <= views AND views < 350 THEN 2
WHEN views < 100 THEN 3
END AS groups
FROM stackoverflow.users
WHERE views > 0 AND location LIKE '%Canada%'),

m AS 
(SELECT id, views, groups, MAX(views) OVER(PARTITION BY groups) AS maximum
FROM u)


SELECT id, groups, views 
FROM m
WHERE views=maximum
ORDER BY maximum DESC, id;

12. Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
SELECT EXTRACT(DAY FROM creation_date) AS day, 
    count(id),
    SUM(COUNT(id)) OVER (ORDER BY EXTRACT(DAY FROM creation_date)) AS cumulative_count
FROM stackoverflow.users
WHERE creation_date BETWEEN '2008-11-01' AND '2008-12-01'
--WHERE DATE_TRUNC('DAY', creation_date)::date BETWEEN '2008-11-01' AND '2008-11-30'
GROUP BY day
ORDER BY day;

13. 
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.:

In [ ]:
SELECT DISTINCT u.id,
    MIN(p.creation_date) OVER(PARTITION BY u.id) - u.creation_date
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON u.id=p.user_id;

### Вторая часть

1. Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
SELECT 
    DATE_TRUNC('MONTH', creation_date)::date dt,
    SUM(views_count) AS sum_v
FROM stackoverflow.posts
WHERE DATE_TRUNC('MONTH', creation_date)::date BETWEEN '2008-01-01' AND '2008-12-31'
GROUP BY dt
ORDER BY sum_v DESC;

2. Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT u.display_name,
    COUNT(DISTINCT user_id) AS count_users
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON p.user_id=u.id
WHERE p.post_type_id = 2
AND
(p.creation_date::date BETWEEN u.creation_date::date AND u.creation_date::date + INTERVAL '1 month')
GROUP BY u.display_name
HAVING COUNT(p.post_type_id) > 100
ORDER BY u.display_name;

3. Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
SELECT DATE_TRUNC('MONTH', creation_date)::date AS month,
    count(id)
FROM stackoverflow.posts
WHERE user_id IN
    (SELECT DISTINCT u.id
    FROM stackoverflow.users AS u
    JOIN stackoverflow.posts AS p ON p.user_id=u.id
    WHERE DATE_TRUNC('MONTH', u.creation_date)::date BETWEEN '2008-09-01' AND '2008-09-30'
    AND
    DATE_TRUNC('MONTH', p.creation_date)::date BETWEEN '2008-12-01' AND '2008-12-31') 
GROUP BY month
ORDER BY month DESC;

4. Используя данные о постах, выведите несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумма просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT user_id,
    creation_date,
    views_count,
    SUM(views_count) OVER(PARTITION BY user_id ORDER BY user_id, creation_date)    
FROM stackoverflow.posts
ORDER BY user_id, creation_date;

5. Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
WITH i AS
    (SELECT
        user_id,
        COUNT(DISTINCT creation_date::date) AS count_dt
    FROM stackoverflow.posts
    WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07'
    GROUP BY user_id)

SELECT ROUND(AVG(count_dt),0)
FROM i;

6. На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? 

In [ ]:
WITH t_2 AS 
(SELECT 
    month_number,
    count,
    LAG(count) OVER (ORDER BY month_number) AS previous_count
FROM
(
    SELECT 
        EXTRACT(MONTH FROM creation_date) AS month_number,
        COUNT(creation_date) AS count
    FROM stackoverflow.posts
    WHERE creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'
    GROUP BY month_number
) t_1
ORDER BY month_number)

SELECT 
    month_number,
    count,
    ROUND((count*100::numeric / previous_count - 100),2) AS percentage_change
FROM t_2  

7. Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
SELECT 
    EXTRACT(WEEK FROM p.creation_date) AS week_number,
    MAX(p.creation_date) AS last_post_date
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id = u.id
WHERE p.creation_date::date BETWEEN '2008-10-01' AND '2008-10-31'
    AND p.user_id = (
        SELECT user_id
        FROM (
            SELECT p2.user_id,
                COUNT(p2.creation_date) AS count
            FROM stackoverflow.posts AS p2
            JOIN stackoverflow.users AS u2 ON p2.user_id = u2.id
            GROUP BY p2.user_id
            ORDER BY count DESC
            LIMIT 1
        ) AS subquery
    )
GROUP BY EXTRACT(WEEK FROM p.creation_date);